<h1>언어 인지</h1>

In [ ]:
# # huggingface 설치: 사전학습 모델 사용을 위한 라이브러리
# !pip install transformers
# !pip install datasets

In [1]:
# 라이브러리
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# IMBD dataset: 영화리뷰 데이터셋
from datasets import load_dataset
data = load_dataset('imdb')
xtrain, ytrain = list(data['train']['text']), list(data['train']['label'])
xtest, ytest = list(data['test']['text']), list(data['test']['label'])

c:\Users\Suhyun\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Suhyun\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Suhyun\.cache\huggingface\hub\datasets--imdb. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. 

# 탐색적 데이터 분석

In [3]:
print( len(xtrain), len(xtest) )

25000 25000


In [4]:
xtrain[0]

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [5]:
ytrain[0]

0

In [6]:
xtrain[13000]

"Visually stunning and full of Eastern Philosophy, this amazing martial arts fantasy is brought to you by master director Tsui Hark, the man behind some of the best films Hong Kong cinema has produced. The special effects are beautiful and imaginative. The plot is a bit on the cerebral side, but is a refreshing change from films that treat their audience as if they were morons. If thinking is not your forte, however, this may not be your movie. Maybe you should go see the latest from the Hollywood studio's no brain club, but if you are looking for something more, he's where you will find it."

In [7]:
ytrain[13000]

1

In [8]:
np.unique(ytrain,return_counts=True)

(array([0, 1]), array([12500, 12500]))

# 텐서플로우를 이용한 감성분석

텍스트 벡터화 (One-hot Encoding)

In [9]:
vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens = 10000,            # number of tokens
    output_sequence_length = 64    # maximum length of sequence
)
vectorize_layer.adapt(xtrain)

In [ ]:
vectorize_layer('I love you')  #10/116/23번째가 1이다!

<tf.Tensor: shape=(64,), dtype=int64, numpy=
array([ 10, 116,  23,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])>

모델링

In [ ]:
model = tf.keras.Sequential()
model.add( vectorize_layer )
model.add( tf.keras.layers.Embedding(10000,32) )  # 10000x32 임베팅 행렬
model.add( tf.keras.layers.SimpleRNN(16) )        # 은닉정보 벡터크기 = 16
model.add( tf.keras.layers.Dense(2,activation='softmax') )  # 문장 분류
# 파라미터 개수: 16*(16+32+1)=784
model.compile()

In [12]:
# 기존 강의 영상 이후 tensorflow 및 라이브러리들의 버전 업데이트에 따라 코드 오류 발생.
# build를 위하여 dummy batch로 한 번 예측.
_ = model(tf.constant([[""]], dtype=tf.string))

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ (1, 64)                │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (1, 64, 32)            │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (1, 16)                │           784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (1, 2)                 │            34 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 320,818 (1.22 MB)

 Trainable params: 320,818 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

모델훈련

In [13]:
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

In [14]:
# 기존 강의 영상 이후 tensorflow 및 라이브러리들의 버전 업데이트에 따라 코드 수정
# 데이터를 tensor 형태로 변환

xtrain = tf.convert_to_tensor(xtrain, dtype=tf.string)
ytrain = tf.convert_to_tensor(ytrain)
xtest = tf.convert_to_tensor(xtest, dtype=tf.string)
ytest = tf.convert_to_tensor(ytest)

In [15]:
model.fit(xtrain,ytrain,epochs=5)

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.5438 - loss: 0.6846
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.7604 - loss: 0.5035
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9008 - loss: 0.2541
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9702 - loss: 0.0945
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9852 - loss: 0.0495


평가 및 테스트

In [16]:
model.evaluate(xtest,ytest)

782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6874 - loss: 1.0836


[1.083603024482727, 0.6874399781227112]

In [17]:
model.predict(tf.convert_to_tensor(['I love this movie', 'Very terrible movie']))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step


array([[0.00966052, 0.99033946],
       [0.9732579 , 0.02674214]], dtype=float32)

# 트랜스포머

In [2]:
from transformers import pipeline

c:\Users\Suhyun\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


문장 분류

In [4]:
classifier = pipeline(task='text-classification')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
c:\Users\Suhyun\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Suhyun\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administr

In [5]:
classifier(['I love this movie', 'Very terrible movie'])

[{'label': 'POSITIVE', 'score': 0.9998766183853149},
 {'label': 'NEGATIVE', 'score': 0.9996885061264038}]

문장 생성

In [ ]:
generator = pipeline(task='text-generation')

No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
generator('In this course, I will teach you how to', max_length=30)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=30) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': 'In this course, I will teach you how to use your smartphone as a social device.\n\nWe will start with the basics of using your smartphone as an Android device.\n\nNext, I will share the basic concept of using your smartphone as a social device.\n\nYou can find all the basic concepts in the Android Tutorials or the Android Tutorials on the Google Play Store. Each app has a tutorial, and each of them has a link to the Android Tutorials.\n\nThe first app has a simple and easy to understand design, and you can read about the design in the Android Tutorials. The app is also available to download.\n\nThe first app has a simple and easy to understand design, and you can read about the design in the Android Tutorials. The app is also available to download. The first app has a simple and easy to understand design, and you can read about the design in the Android Tutorials.\n\nThe second app has a simple and easy to understand design, and you can read about the design in the

In [ ]:
# 한국어 문장 생성
kor_generator = pipeline("text-generation","skt/kogpt2-base-v2")

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
kor_generator("오늘 야구 경기에서는 ", max_length=30)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=30) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': '오늘 야구 경기에서는 넥센 히어로즈 소속의 넥센 히어로즈 내야수 박민우의 선방에 막히고 말았다.\n당시 이 경기에선 넥센 히어로즈가 내야수로 선발출전해 3루타를 쳐내는 등 무실점의 경기가 이어졌다.\n당시 투수진은 1루수 정병욱의 내야안타의 맹공을 막아낼 수 있었다.\n하지만 이 무실점은 투수들의 무실점으로 이어지지 않았다.\n1루수 정병욱은 1타점 적시타를 쳐내며 타격에 성공, 2타점 적시타와 볼넷을 얻었다.\n이렇듯 넥센의 무실점으로 넥센팬들의 기대를 저버린 투수였다.\n당시 야구는 넥센 히어로즈 소속 선수의 무대였다.\n당시 넥센은 경기 내내 3루타를 치며 타격감을 끌어올렸지만 무실점으로 이어진 것은 이례적이었다.\n넥센은 2012년 시즌부터 2014년 시즌까지 시즌 평균자책점이 0.78에 그치고 있는 상황이었다.\n이 때문에 2017년 시즌부터는 타격감이 떨어지기 시작했다.\n2017년 시즌부터 넥센은 경기마다 3루타와 볼넷, 홈런을 허용했다.\n이러한 상황은 한화 이글스의 강민호와 박병호의 활약에 영향을 주었다.\n한화 이글스의 주전 투수였던 박병'}]

빈칸 채우기

In [ ]:
# 빈칸 채우기
unmasker = pipeline("fill-mask")

No model was supplied, defaulted to distilbert/distilroberta-base and revision fb53ab8 (https://huggingface.co/distilbert/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
unmasker("This course will teach you all about <mask> models.", top_k=2)

[{'score': 0.19619743525981903,
  'token': 30412,
  'token_str': ' mathematical',
  'sequence': 'This course will teach you all about mathematical models.'},
 {'score': 0.04052695631980896,
  'token': 38163,
  'token_str': ' computational',
  'sequence': 'This course will teach you all about computational models.'}]

Q&A

In [ ]:
question_answerer = pipeline("question-answering")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
question_answerer(
    question="Where do I study?",
    context="My name is Jaewon and I am a student of Korea University"
)

{'score': 0.9710100889205933,
 'start': 40,
 'end': 56,
 'answer': 'Korea University'}

# 연습문제

1. 위의 IMBD 데이터를 학습하는 모델에서 SImpleRNN 모듈 대신에 GRU 모듈을 사용하며, 은닉정보 벡터크기를 32로 변경하여 모델을 구성하고 학습하시오.
```tf.keras.layers.GRU```
를 사용하시오. <br>
(1) 모델의 학습가능한 파라메터(Trainable params)의 수는 몇 개인가? <br>
(2) 학습 후 평가데이터의 정확도는 얼마인가? <br>



In [10]:
# 라이브러리
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [11]:
# IMBD dataset: 영화리뷰 데이터셋
from datasets import load_dataset
data = load_dataset('imdb')
xtrain, ytrain = list(data['train']['text']), list(data['train']['label'])
xtest, ytest = list(data['test']['text']), list(data['test']['label'])

In [12]:
vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens = 10000,            # number of tokens
    output_sequence_length = 64    # maximum length of sequence
)
vectorize_layer.adapt(xtrain)

In [13]:
model = tf.keras.Sequential()
model.add( vectorize_layer )
model.add( tf.keras.layers.Embedding(10000,32) )  # 10000x32 임베팅 행렬
model.add( tf.keras.layers.GRU(32) )        # 은닉정보 벡터크기 = 32
model.add( tf.keras.layers.Dense(2,activation='softmax') )  # 문장 분류
model.compile()

In [14]:
_ = model(tf.constant([[""]], dtype=tf.string))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (Text  (None, 64)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 64, 32)            320000    
                                                                 
 gru_1 (GRU)                 (None, 32)                6336      
                                                                 
 dense_1 (Dense)             (None, 2)                 66        
                                                                 
Total params: 326402 (1.25 MB)
Trainable params: 326402 (1.25 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2. HuggingFace의 pipeline을 사용하여 문장 분류를 수행하시오. 기본적인 모델을 사용하였을 때, "I ate vongole pasta."에 대한 긍정 확률은 얼마인가?

In [ ]:
from transformers import pipeline
classifier = pipeline(task='text-classification')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [21]:
classifier(['I ate vongole pasta.'])

[{'label': 'NEGATIVE', 'score': 0.5121523141860962}]

In [22]:
1-0.5121523141860962

0.4878476858139038